# S3 Video Memory Tool Demo
### Intelligent Conversation History with JSON Classification and S3 Vector Storage

This notebook demonstrates how to use the S3 Video Memory tool - a mem0_memory-like interface that combines automatic JSON classification with S3 vector storage for intelligent conversation management.

**Key Features:**
- **Similar to mem0_memory**: store, retrieve, list, delete actions
- **Automatic JSON Classification**: Content classified into structured JSON with categories, keywords, sentiment, etc.
- **S3 Vector Backend**: Uses S3 Vectors instead of OpenSearch/FAISS/Mem0
- **User-Specific Filtering**: Filter conversations by user_id or agent_id
- **Semantic Search**: Vector-based similarity search using classified summaries
- **Rich Console Output**: Beautiful formatting with confirmation dialogs
- **Smart Embeddings**: Generates embeddings from classified summaries for better search

**Prerequisites:**
- AWS credentials configured
- Amazon S3 Vector bucket created
- Amazon Bedrock access enabled (for embeddings and classification)
- Strands agent with S3 Video Memory tool loaded

## 1. Setup and Configuration

First, let's set up our environment and configure the necessary parameters.

In [1]:
# Import necessary libraries
import boto3
import os
import json
from datetime import datetime
from strands.models import BedrockModel
from strands import Agent
from s3_video_memory import s3_video_memory

AWS_REGION = 'us-east-2'

# Environment configuration
os.environ['VECTOR_BUCKET_NAME'] = 'my-vector-bucket-1bf5-fkxn8xdmekn3'# Your S3 vector bucket name
os.environ['AWS_REGION'] = AWS_REGION  # Your preferred AWS region
os.environ['VECTOR_INDEX'] = 'conversation-index'  # Your preferred AWS region
os.environ['BYPASS_TOOL_CONSENT'] = 'true'  # Skip confirmations for demo

# Configure AWS session for Bedrock access
session = boto3.Session(region_name=AWS_REGION)

# Initialize Bedrock model for inference
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Initialize Strands agent with S3 Video Memory tool
agent = Agent(
    tools=[s3_video_memory],
    system_prompt="You are a helpful assistant with intelligent conversation memory using S3 Video Memory.",
    model=bedrock_model
)

print("✅ Environment configured successfully!")
print(f"Vector Bucket: {os.environ['VECTOR_BUCKET_NAME']}")
print(f"AWS Region: {os.environ['AWS_REGION']}")
print(f"Bypass Consent: {os.environ['BYPASS_TOOL_CONSENT']}")

✅ Environment configured successfully!
Vector Bucket: my-vector-bucket-1bf5-fkxn8xdmekn3
AWS Region: us-east-2
Bypass Consent: true


## 2. Storing Conversations with Automatic Classification

Let's demonstrate how the tool automatically classifies content into JSON structure and stores it with user-specific metadata.

In [3]:
# Sample conversation content for different users
conversations = [
    {
        "user_id": "alice123",
        "content": "Usuario: ¿Qué es el machine learning? Asistente: Machine learning es una rama de la inteligencia artificial que permite a las computadoras aprender y mejorar automáticamente a partir de la experiencia sin ser programadas explícitamente. Se basa en algoritmos que pueden identificar patrones en datos y hacer predicciones.",
        "session_id": "session_001"
    },
    {
        "user_id": "alice123",
        "content": "Usuario: ¿Cómo funcionan las redes neuronales? Asistente: Las redes neuronales son modelos computacionales inspirados en el funcionamiento del cerebro humano. Están compuestas por nodos interconectados (neuronas artificiales) que procesan información a través de conexiones ponderadas, permitiendo el aprendizaje de patrones complejos.",
        "session_id": "session_002"
    },
    {
        "user_id": "bob456",
        "content": "Usuario: ¿Qué es Python? Asistente: Python es un lenguaje de programación de alto nivel, interpretado y de propósito general. Se caracteriza por su sintaxis clara y legible, lo que lo hace ideal para principiantes. Es ampliamente utilizado en desarrollo web, ciencia de datos, automatización y machine learning.",
        "session_id": "session_003"
    },
    {
        "user_id": "bob456",
        "content": "Usuario: ¿Cómo creo una función en Python? Asistente: Para crear una función en Python, usa la palabra clave 'def' seguida del nombre de la función y parámetros. Por ejemplo: def saludar(nombre): return f'Hola, {nombre}!'. Las funciones son bloques de código reutilizables que realizan tareas específicas.",
        "session_id": "session_004"
    },
    {
        "user_id": "carlos789",
        "content": "Usuario: ¿Qué es la computación en la nube? Asistente: La computación en la nube es la entrega de servicios informáticos (servidores, almacenamiento, bases de datos, redes, software) a través de internet. Permite acceso bajo demanda, escalabilidad y pago por uso, eliminando la necesidad de infraestructura física local.",
        "session_id": "session_005"
    }
]

# Store each conversation with automatic JSON classification
print("🔄 Storing conversations with automatic classification...\n")

stored_memories = []
for i, conv in enumerate(conversations):
    print(f"📝 Storing conversation {i+1} for user {conv['user_id']}...")
    
    result = agent.tool.s3_video_memory(
        action="store",
        content=conv["content"],
        user_id=conv["user_id"],
        metadata={
            "session_id": conv["session_id"],
            "stored_at": datetime.now().isoformat()
        }
    )
    print(result)
    
    if result["status"] == "success":
        print(f"✅ Successfully stored conversation {i+1}")
        stored_memories.append(result)
    else:
        print(f"❌ Failed to store conversation {i+1}: {result.get('content', [{}])[0].get('text', 'Unknown error')}")
    
    print("-" * 50)

print(f"\n🎉 Stored {len(stored_memories)} conversations successfully!")
print("\n📊 Each conversation was automatically classified with:")
print("   - Summary (used for embedding generation)")
print("   - Category (conversacion, pregunta, informacion, etc.)")
print("   - Keywords extracted from content")
print("   - Sentiment analysis")
print("   - Topics identification")
print("   - Importance level")
print("   - Context and intent analysis")

🔄 Storing conversations with automatic classification...

📝 Storing conversation 1 for user alice123...
{'summary': 'Usuario: ¿Qué es el machine learning? Asistente: Machine learning es una rama de la inteligencia artificial que permite a las computadoras aprender y mejorar automáticamente a partir de la experiencia...', 'category': 'otro', 'keywords': [], 'sentiment': 'neutral', 'topics': [], 'importance': 'media', 'context': 'contenido sin clasificar', 'intent': 'desconocido'}


╭─────────────────────────────────────── Memory to store for user alice123 ───────────────────────────────────────╮
│ Usuario: ¿Qué es el machine learning? Asistente: Machine learning es una rama de la inteligencia artificial que │
│ permite a las computadoras aprender y mejorar automáticamente a partir de la experiencia sin ser programadas    │
│ explícitamente. Se basa en algoritmos que pueden identificar patrones en datos y hacer predicciones.            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_s3_video_memory_326842863', 'status': 'success', 'content': [{'text': 'None'}]}
✅ Successfully stored conversation 1
--------------------------------------------------
📝 Storing conversation 2 for user alice123...
{'summary': 'Usuario: ¿Cómo funcionan las redes neuronales? Asistente: Las redes neuronales son modelos computacionales inspirados en el funcionamiento del cerebro humano. Están compuestas por nodos interconectado...', 'category': 'otro', 'keywords': [], 'sentiment': 'neutral', 'topics': [], 'importance': 'media', 'context': 'contenido sin clasificar', 'intent': 'desconocido'}


╭─────────────────────────────────────── Memory to store for user alice123 ───────────────────────────────────────╮
│ Usuario: ¿Cómo funcionan las redes neuronales? Asistente: Las redes neuronales son modelos computacionales      │
│ inspirados en el funcionamiento del cerebro humano. Están compuestas por nodos interconectados (neuronas        │
│ artificiales) que procesan información a través de conexiones ponderadas, permitiendo el aprendizaje de         │
│ patrones complejos.                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_s3_video_memory_541518477', 'status': 'success', 'content': [{'text': 'None'}]}
✅ Successfully stored conversation 2
--------------------------------------------------
📝 Storing conversation 3 for user bob456...
{'summary': 'Usuario: ¿Qué es Python? Asistente: Python es un lenguaje de programación de alto nivel, interpretado y de propósito general. Se caracteriza por su sintaxis clara y legible, lo que lo hace ideal para ...', 'category': 'otro', 'keywords': [], 'sentiment': 'neutral', 'topics': [], 'importance': 'media', 'context': 'contenido sin clasificar', 'intent': 'desconocido'}


...

{'toolUseId': 'tooluse_s3_video_memory_363515201', 'status': 'success', 'content': [{'text': 'None'}]}
✅ Successfully stored conversation 3
--------------------------------------------------
📝 Storing conversation 4 for user bob456...
{'summary': "Usuario: ¿Cómo creo una función en Python? Asistente: Para crear una función en Python, usa la palabra clave 'def' seguida del nombre de la función y parámetros. Por ejemplo: def saludar(nombre): retu...", 'category': 'otro', 'keywords': [], 'sentiment': 'neutral', 'topics': [], 'importance': 'media', 'context': 'contenido sin clasificar', 'intent': 'desconocido'}


╭──────────────────────────────────────── Memory to store for user bob456 ────────────────────────────────────────╮
│ Usuario: ¿Cómo creo una función en Python? Asistente: Para crear una función en Python, usa la palabra clave    │
│ 'def' seguida del nombre de la función y parámetros. Por ejemplo: def saludar(nombre): return f'Hola,           │
│ {nombre}!'. Las funciones son bloques de código reutilizables que realizan tareas específicas.                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_s3_video_memory_997316979', 'status': 'success', 'content': [{'text': 'None'}]}
✅ Successfully stored conversation 4
--------------------------------------------------
📝 Storing conversation 5 for user carlos789...
{'summary': 'Usuario: ¿Qué es la computación en la nube? Asistente: La computación en la nube es la entrega de servicios informáticos (servidores, almacenamiento, bases de datos, redes, software) a través de inter...', 'category': 'otro', 'keywords': [], 'sentiment': 'neutral', 'topics': [], 'importance': 'media', 'context': 'contenido sin clasificar', 'intent': 'desconocido'}


╭────────────────────────────────────── Memory to store for user carlos789 ───────────────────────────────────────╮
│ Usuario: ¿Qué es la computación en la nube? Asistente: La computación en la nube es la entrega de servicios     │
│ informáticos (servidores, almacenamiento, bases de datos, redes, software) a través de internet. Permite acceso │
│ bajo demanda, escalabilidad y pago por uso, eliminando la necesidad de infraestructura física local.            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_s3_video_memory_648814704', 'status': 'success', 'content': [{'text': 'None'}]}
✅ Successfully stored conversation 5
--------------------------------------------------

🎉 Stored 5 conversations successfully!

📊 Each conversation was automatically classified with:
   - Summary (used for embedding generation)
   - Category (conversacion, pregunta, informacion, etc.)
   - Keywords extracted from content
   - Sentiment analysis
   - Topics identification
   - Importance level
   - Context and intent analysis


## 3. Semantic Search with User Filtering

Now let's demonstrate how to search for relevant conversations using semantic search, filtered by specific users.

In [7]:
def search_user_conversations(user_id: str, query: str, top_k: int = 3):
    """
    Search for conversations specific to a user using semantic search
    
    Args:
        user_id: The user ID to filter conversations
        query: The search query
        top_k: Number of results to return
    """
    print(f"\n🔍 Searching conversations for user '{user_id}' with query: '{query}'")
    print(f"   Top {top_k} results, minimum score: 0.7")
    
    result = agent.tool.s3_video_memory(
        action="retrieve",
        query=query,
        user_id=user_id,
        top_k=top_k,
        min_score=0.7
    )
    
    if result["status"] == "success":
        print(f"✅ Search completed successfully")
        print(f"   Results are displayed above with rich formatting")
        
        # Parse the JSON response to show additional details
        try:
            memories = json.loads(result["content"][0]["text"])
            print(f"\n📈 Search Analytics:")
            print(f"   - Total results found: {len(memories)}")
            
            if memories:
                avg_score = sum(m["score"] for m in memories) / len(memories)
                print(f"   - Average relevance score: {avg_score:.3f}")
                print(f"   - Best match score: {max(m["score"] for m in memories):.3f}")
                
                categories = [m["metadata"]["category"] for m in memories if m["metadata"].get("category")]
                if categories:
                    print(f"   - Categories found: {', '.join(set(categories))}")
            else:
                print(f"   - No results met the minimum score threshold")
                
        except (json.JSONDecodeError, KeyError) as e:
            print(f"   - Could not parse detailed analytics: {e}")
    else:
        print(f"❌ Search failed: {result.get('content', [{}])[0].get('text', 'Unknown error')}")
    
    return result



In [2]:
# Test semantic search for different users and topics
search_scenarios = [
    ("alice123", "inteligencia artificial y aprendizaje automático"),
    ("alice123", "redes neuronales y deep learning"),
    ("bob456", "programación en Python"),
    ("bob456", "crear funciones y código"),
    ("carlos789", "servicios en la nube"),
    ("alice123", "algoritmos de machine learning"),  # Cross-topic search
    ("bob456", "desarrollo de software"),  # Broader search
]

print("🎯 Testing semantic search scenarios...\n")

for user_id, query in search_scenarios:
    search_user_conversations(user_id, query, top_k=2)
    print("\n" + "="*80 + "\n")

🎯 Testing semantic search scenarios...



NameError: name 'search_user_conversations' is not defined

## 4. Context-Aware Response Generation

Let's demonstrate how to use the stored conversation history to generate context-aware responses.

In [ ]:
def generate_context_aware_response(user_id: str, new_query: str):
    """
    Generate a context-aware response using conversation history
    
    Args:
        user_id: The user ID
        new_query: The new query from the user
    """
    print(f"\n🤖 Generating context-aware response for user: {user_id}")
    print(f"   New query: '{new_query}'")
    
    # Step 1: Search for relevant conversation history
    print(f"\n🔍 Searching for relevant conversation history...")
    
    history_result = agent.tool.s3_video_memory(
        action="retrieve",
        query=new_query,
        user_id=user_id,
        top_k=3,
        min_score=0.6
    )
    
    context_available = False
    context_summary = ""
    
    if history_result["status"] == "success":
        try:
            memories = json.loads(history_result["content"][0]["text"])
            
            if memories:
                context_available = True
                print(f"   ✅ Found {len(memories)} relevant past conversations")
                
                # Build context summary
                context_items = []
                for i, memory in enumerate(memories, 1):
                    metadata = memory["metadata"]
                    context_items.append(f"{i}. {metadata.get('category', 'N/A')} - {memory['memory'][:100]}...")
                    
                context_summary = "\n".join(context_items)
                
                print(f"\n   📚 Context Summary:")
                for memory in memories:
                    metadata = memory["metadata"]
                    print(f"      - Category: {metadata.get('category')}")
                    print(f"        Keywords: {', '.join(metadata.get('keywords', []))}")
                    print(f"        Relevance: {memory['score']:.3f}")
                    print(f"        Summary: {memory['memory'][:80]}...")
                    print()
            else:
                print(f"   ℹ️ No relevant conversation history found")
        except Exception as e:
            print(f"   ❌ Error processing history: {e}")
    else:
        print(f"   ❌ Failed to search history")
    
    # Step 2: Generate response strategy
    print(f"\n💡 Response Strategy:")
    
    if context_available:
        print(f"   🎯 Context-aware response approach:")
        print(f"      - Reference previous discussions")
        print(f"      - Build upon existing knowledge")
        print(f"      - Provide advanced or related information")
        print(f"      - Use personalized examples")
        
        # Simulate storing the context-aware response
        response_content = f"Usuario: {new_query} Asistente: [Respuesta contextual basada en historial previo sobre {', '.join(set(m['metadata'].get('category', 'N/A') for m in memories))}]"
    else:
        print(f"   🌱 Fresh start approach:")
        print(f"      - Provide comprehensive introduction")
        print(f"      - Start with fundamental concepts")
        print(f"      - Offer to explain basics")
        print(f"      - Suggest related topics")
        
        # Simulate storing the fresh response
        response_content = f"Usuario: {new_query} Asistente: [Respuesta completa e introductoria]"
    
    # Step 3: Store the new conversation
    print(f"\n💾 Storing new conversation...")
    
    store_result = agent.tool.s3_video_memory(
        action="store",
        content=response_content,
        user_id=user_id,
        metadata={
            "has_context": "yes" if context_available else "no",
            "context_count": len(memories) if context_available else 0,
            "query_type": "follow_up" if context_available else "initial",
            "session_id": f"demo_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        }
    )
    
    if store_result["status"] == "success":
        print(f"   ✅ New conversation stored successfully")
    else:
        print(f"   ❌ Failed to store new conversation")
    
    return {
        "context_available": context_available,
        "context_summary": context_summary,
        "response_stored": store_result["status"] == "success"
    }

# Test context-aware responses
context_test_scenarios = [
    ("alice123", "¿Qué es deep learning?"),  # Should find ML context
    ("alice123", "¿Cómo se entrenan los modelos?"),  # Should find neural network context
    ("bob456", "¿Qué son las librerías de Python?"),  # Should find Python context
    ("bob456", "¿Cómo hacer debugging?"),  # Should find programming context
    ("carlos789", "¿Qué es DevOps?"),  # Should find cloud context
    ("carlos789", "¿Qué es blockchain?"),  # No context - new topic
]

print("🎯 Testing context-aware response generation...\n")

for user_id, query in context_test_scenarios:
    result = generate_context_aware_response(user_id, query)
    print(f"\n📊 Result: Context {'available' if result['context_available'] else 'not available'}")
    print("\n" + "="*80 + "\n")